In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#hla types for all samples
file_path = "hla_DQ_genotypes.csv"

In [ ]:
#updated hla-DQ types
file_path2 = "20240628_HLA_hk.csv"

In [ ]:
data= pd.read_csv(file_path)

In [ ]:
DQ_data= pd.read_csv(file_path2)

In [ ]:
allele_counts = {}

In [ ]:
#calculate the frequency of HLA types for each gene for cases and controls
def gene_freq(gene):
    gene_df=pd.concat([data[['Celiac',f'{gene}.1']].rename({f'{gene}.1':gene},axis=1),data[['Celiac',f'{gene}.2']].rename({f'{gene}.2':gene},axis=1)])
    if gene=='DQA1':
        gene_df=pd.concat([DQ_data[['CeD','hibag.DQA1.1 update']].rename({'hibag.DQA1.1 update':gene},axis=1),DQ_data[['CeD','hibag.DQA1.2 update']].rename({'hibag.DQA1.2 update':gene},axis=1)])
        gene_df.rename({'CeD':'Celiac'},axis=1,inplace=True)
    print(len(gene_df))
    gene_df2=gene_df.value_counts()
    print(gene_df2)
    count=pd.DataFrame(gene_df2)
    count.reset_index(inplace=True)
    final_ct=count.pivot(gene,'Celiac',0).fillna(0)
    final_ct['ctrl %']=final_ct[1]/9457/2
    final_ct['CeD %']=final_ct[2]/1930/2
    final_ct['delta']=final_ct['CeD %']-final_ct['ctrl %']
    final_ct.sort_values('delta',ascending=False,inplace=True)
    final_ct['gene']=gene
    return(final_ct)


In [ ]:
genes = ['A','B','C','DPB1','DQA1','DQB1','DRB1']
gene_dfs=[]

for gene in genes:
    df = gene_freq(gene)
    gene_dfs.append(df)
    
final_gene_df=pd.concat(gene_dfs)
final_gene_df

In [ ]:
#only select those which are more prevalent in CeD for analysis
enrich_thresh=0
enriched=final_gene_df[final_gene_df['delta']>enrich_thresh]
other=final_gene_df[final_gene_df['delta']<=enrich_thresh]
#84 with positive delta

In [ ]:
#chi square and OR; use those with genotypes not enriched as unexposed group
alt_counts=other.groupby('gene').agg('sum')
alt_counts['delta']=alt_counts['CeD %']-alt_counts['ctrl %']

In [ ]:
alt_counts.to_csv('alternative_counts_per_gene_chisq.csv')

In [ ]:
enriched.index=enriched['gene']+'*'+enriched.index

In [ ]:
from scipy.stats import chi2_contingency
import statsmodels.stats.multitest as st
#chi square and odds ratio
for i,row in enriched.iterrows():
    obs=np.array([[row[1],row[2]],[alt_counts.loc[row['gene'],1],alt_counts.loc[row['gene'],2]]])
    res=chi2_contingency(obs)
    enriched.loc[i,'X^2']=res.statistic
    enriched.loc[i,'pvalue']=res.pvalue
    enriched.loc[i,'OR univar']=row[2]*alt_counts.loc[row['gene'],1]/(row[1]*alt_counts.loc[row['gene'],2])
enriched.sort_values('OR univar')

In [ ]:
#FDR correction
def FDR(sample, pval):
    sample.sort_values(by=pval, inplace = True)
    samp = sample.dropna(subset=[pval])
    pvals = samp[pval]
    padj = st.fdrcorrection(pvals, is_sorted=True)
    padded_col = padj[1]
    sample['Padj'] = padded_col

FDR(enriched,'pvalue')


In [ ]:
enriched.to_csv("enriched_hla.csv")

In [ ]:
#use 0.005 as significance cutoff, we will only run logistic regression for those with p<0.005
pval_thresh=0.005
sig_enriched=enriched[enriched['Padj']<pval_thresh]
#18 hla types

In [ ]:
risk_alleles = sig_enriched.index

In [ ]:
for risk_allele in risk_alleles:
    # Split the risk allele into gene and allele
    gene, allele = risk_allele.split('*')
    
    # Count occurrences in both columns for the gene
    allele_count = (data[f'{gene}.1'] == allele).sum() + (data[f'{gene}.2'] == allele).sum()
    if gene=='DQA1':
        allele_count = (DQ_data[f'hibag.{gene}.1 update'] == allele).sum() + (DQ_data[f'hibag.{gene}.2 update'] == allele).sum()
    # Store the count in the dictionary
    allele_counts[risk_allele] = allele_count

In [ ]:
allele_counts_by_status = {}

In [ ]:
allele_counts_df = pd.DataFrame(index=risk_alleles)

In [ ]:
celiac_statuses = data['Celiac'].unique()

In [ ]:
DQ_data.rename({'person_id':'sample.id'},axis=1,inplace=True)

In [ ]:
data=pd.merge(data,DQ_data[['sample.id','hibag.DQA1.1 update','hibag.DQA1.2 update']],on='sample.id')
data.drop(['DQA1.1','DQA1.2'],axis=1,inplace=True)
data.rename({'hibag.DQA1.1 update':'DQA1.1','hibag.DQA1.2 update':'DQA1.2'},axis=1,inplace=True)

In [ ]:
for status in celiac_statuses:
    # Filter data for the current Celiac status
    data_status = data[data['Celiac'] == status]
    
    # Initialize a list to store counts for the current status
    counts_for_status = []
    
    # Iterate over each risk allele
    for risk_allele in risk_alleles:
        # Split the risk allele into gene and allele
        gene, allele = risk_allele.split('*')
        # Count occurrences in both columns for the gene
        allele_count = (data_status[f'{gene}.1'] == allele).sum() + (data_status[f'{gene}.2'] == allele).sum()
        # Append the count to the list
        counts_for_status.append(allele_count)
    
    # Add the counts to the DataFrame under a column named after the current status
    allele_counts_df[status] = counts_for_status

In [ ]:
allele_counts_df.to_csv('allele_cts_20240705.csv')

In [ ]:
risk_alleles = sig_enriched.index

In [ ]:
for risk_allele in risk_alleles:
    gene, allele = risk_allele.split('*')
    #Dominant model; at least one allele as a case
    data[f'{risk_allele}_dummy'] = ((data[f'{gene}.1'] == allele) | (data[f'{gene}.2'] == allele)).astype(int)

In [ ]:
additional_covariates = ['age', 'sex'] + [f'PC{i}' for i in range(1, 16)]

In [ ]:
#logistic regression; run all hlatypes as covariates with each other
X = data[additional_covariates + [f'{allele}_dummy' for allele in risk_alleles]]

In [ ]:
X = sm.add_constant(X) 

In [ ]:
y = data['Celiac'].map({1: 0, 2: 1}) 

In [ ]:
model = sm.Logit(y, X).fit(method='bfgs',maxiter=1000)

In [ ]:
summary_str = model.summary().as_text()

In [ ]:
output_path = 'logistic regression5.csv'

In [ ]:
with open(output_path, 'w') as file:
    file.write(summary_str)

print(f"Summary saved to {output_path}")

In [ ]:
model.summary()

In [ ]:
X.columns

In [ ]:
summary_table = model.summary2().tables[1]

In [ ]:
summary_table['P>|z|'] = pd.to_numeric(summary_table['P>|z|'], errors='coerce')
summary_table['P>|z|'].apply(lambda x: '{:.3f}'.format(x) if pd.notnull(x) else x)
print(summary_table.head())

In [ ]:
print(summary_table)

In [ ]:
output_path = "Regression_results_20240705.csv"
summary_table.to_csv(output_path, index=True)
print(f"summary_table saved to: {output_path}")

In [ ]:
#calculate adjusted OR from beta
coefficients = model.params
conf = model.conf_int()
conf['OR'] = coefficients
conf.columns = ['2.5%', '97.5%', 'OR']
conf = np.exp(conf)
conf = conf.round(3)
conf['97.5%'] = conf['97.5%'].apply(lambda x: '{:.3f}'.format(x))

In [ ]:
conf.columns=['Log-OR 0.025','Log-OR 0.075','Log-OR']

In [ ]:
log_result=pd.concat([summary_table,conf],axis=1)

In [ ]:
log_result.index=log_result.index.str.replace('_dummy','')
log_result

In [ ]:
#final cleaning before saving file
log_result_final=pd.concat([sig_enriched,log_result],axis=1)
log_result_final.drop('gene',axis=1,inplace=True)
log_result_final=log_result_final[~pd.isna(log_result_final[1])]
log_result_final[['ctrl %','CeD %']]=log_result_final[['ctrl %','CeD %']]*100
log_result_final.rename({1:'non-CeD',2:'CeD','ctrl %':'non-CeD %'},axis=1)
log_result_final.sort_values('Log-OR',ascending=False,inplace=True)

In [ ]:
log_result_final.to_csv('logistic_result_20240705.csv')

In [ ]:
conf.to_csv("Regression_results_20240701_OR.csv", index=True)